# 위상 정렬  

방향 비순환 그래프(DAG)에서 선행 조건(의존 관계)을 만족하면서 모든 노드를 순서대로 나열하는 알고리즘이다.

진입 차수(in degree)는 선행 조건을 의미한다. 즉 선행 조건인 in degree가 0을 기준으로 정렬을 수행한다.

- 기본 위상정렬: 진입 차수 0인 노드를 큐에 넣어 순서대로 처리 (Kahn’s Algorithm)
- 사전순 정렬: 진입 차수 0인 노드 중 번호가 작은 순으로 처리 (우선순위 큐 사용)
- 사이클 탐지: DFS나 위상 정렬 도중 순환 구조가 있는지 판단
- 동적 간선 변경: 간선 정보를 실시간으로 수정한 뒤 위상 정렬 수행


In [1]:
n = 6
graph = [
  [],      # 0번 노드: 나가는 간선 없음
  [],      # 1번 노드: 나가는 간선 없음
  [3],     # 2 → 3
  [1],     # 3 → 1
  [0, 1],  # 4 → 0, 1
  [0, 2]   # 5 → 0, 2
]

In [2]:
# 기본 위상정렬 (BFS-queue) - 간단, 빠름
from collections import deque

def topo_sort(n, graph):
  # 1. 진입 차수 계산 : 각 노드의 선행 작업 확인
  in_degree = [0] * n # 선행 작업 개념

  for u in range(n):
    for v in graph[u]:
      in_degree[v] += 1

  # 2. 초기화 : 진입 차수가 0인 노드들을 큐에 넣어 시작점으로 설정
  result = []
  q = deque(i for i in range(n) if in_degree[i] == 0)
  
  # 3. 위상 정렬 : 진입 차수가 0인 노드를 제거하며 순서 생성
  while q:
    # 3.1. 노드 순서 생성
    current = q.popleft()
    result.append(current)

    # 3.2. 연결된 노드들의 진입 차수 감소
    for next in graph[current]:
      in_degree[next] -= 1
      if in_degree[next] == 0:
        q.append(next)

  # 4. 모든 노드를 방문했다면 정렬 결과를 반환, 아니면 사이클 존재
  return result if len(result) == n else None

print("기본 위상 정렬 :", topo_sort(n, graph))

기본 위상 정렬 : [4, 5, 0, 2, 3, 1]


In [3]:
# 사전순(우선순위) 위상정렬 - 결과 일관성, 조건 제어 가능
import heapq

def lex_topo_sort(n, graph):
  # 1. 진입 차수 계산 : 각 노드의 선행 작업 확인
  in_degree = [0] * n
  for u in range(n):
    for v in graph[u]:
      in_degree[v] += 1

  # 2. 초기화 : 진입 차수 0인 노드들을 힙에 삽입 (사전순 자동 정렬)
  result = []
  heap = [i for i in range(n) if in_degree[i] == 0]
  heapq.heapify(heap)

  # 3. 위상 정렬 : 진입 차수가 0인 노드를 제거하며 순서 생성
  while heap:
    # 3.1. 노드 순서 생성
    current = heapq.heappop(heap)
    result.append(current)

    # 3.2. 연결된 노드들의 진입 차수 감소
    for next in graph[current]:
      in_degree[next] -= 1
      if in_degree[next] == 0:
        heapq.heappush(heap, next)

  # 4. 모든 노드를 방문했다면 정렬 결과를 반환, 아니면 사이클 존재
  return result if len(result) == n else None

print('사전순 위상 정렬 :', lex_topo_sort(n, graph))

사전순 위상 정렬 : [4, 5, 0, 2, 3, 1]


In [4]:
# 사이클 탐지
def detect_cycle(n, graph):
  visited = [0] * n # 0: 미방문, 1: 방문 중, 2: 방문 완료

  # DFS로 노드를 방문하며 사이클 존재 여부 확인
  def dfs(node):
    visited[node] = 1

    for next in graph[node]:
      if visited[next] == 2: # 이미 방문 완료된 노드는 건너뜀
        continue
      elif visited[next] == 1: # 방문 중인 노드를 다시 방문하면 사이클 발생
        return True 
      elif visited[next] == 0: # 방문하지 않은 노드라면 DFS로 재귀 탐색
        if dfs(next): # 하위 노드의 사이클 탐지
          return True
  
    visited[node] = 2
    return False
  
  # 모든 노드에 대해 DFS 탐색
  for node in range(n):
    if visited[node] == 0:
      if dfs(node):
        return 'Have cycle'
  return 'No Cycle'
  
print('사이클 존재 여부 :', detect_cycle(n, graph))

사이클 존재 여부 : No Cycle


In [5]:
def dynamic_edge_topo_sort(n, current, changes):
  # 1. 기본 순서로 간선 구성 : u가 v보다 앞에 있다면 u → v 간선 추가
  edge = [[False] * n for _ in range(n)]
  for i in range(n):
    for j in range(i+1, n):
      u, v = current[i], current[j]
      edge[u][v] = True

  # 2. 간선 방향 변경 적용
  for u, v in changes:
    if edge[u][v]:
      edge[u][v], edge[v][u] = False, True
    else:
      edge[u][v], edge[v][u] = True, False

  # 3. 수정된 그래프 및 진입 차수 구성
  graph = [[] for _ in range(n)]
  in_degree = [0] * n
  for u in range(n):
    for v in range(n):
      if edge[u][v]:
        graph[u].append(v)
        in_degree[v] += 1

  # 4. 위상 정렬 수행
  result = []
  queue = deque(i for i in range(n) if in_degree[i] == 0)
  while queue:
    if len(queue) > 1:
      return "AMBIGUOUS"  # 순서를 확정할 수 없음 - 가능한 정렬이 여러개
    
    current = queue.popleft()
    result.append(current)

    for next in graph[current]:
      in_degree[next] -= 1
      if in_degree[next] == 0:
        queue.append(next)

  # 5. 모든 노드를 방문했다면 정렬 결과를 반환, 아니면 사이클 존재
  return result if len(result) == n else "IMPOSSIBLE"

current = [4, 5, 0, 2, 3, 1]
changes = [(5, 0), (3, 1)]
print('동적 간선 변경 결과 :', dynamic_edge_topo_sort(n, current, changes))

동적 간선 변경 결과 : [4, 0, 5, 2, 1, 3]
